<a href="https://colab.research.google.com/github/towardsai/ai-tutor-rag-system/blob/main/notebooks/Firecrawl_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the requirements

In [ ]:
!pip install -q llama-index==0.12.48 llama-index-readers-web==0.3.7 openai==1.107.0 jedi==0.19.2 firecrawl-py==1.15.0

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.0/951.0 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### SET THE ENVIRONMENT VARIABLES

In [ ]:
import os
# os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
# FIRECRAWL_API_KEY = "<FIRECRAWL_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["FIRECRAWL_API_KEY"] = userdata.get('FIRECRAWL_API_KEY')

FIRECRAWL_API_KEY = userdata.get('FIRECRAWL_API_KEY')

# SCRAPE WITH FIRECRAWL

## IMPORT THE FIRECRAWL WEBREADER

Firecrawl allows you to turn entire websites into LLM-ready markdown

Get the API key here
https://www.firecrawl.dev/app/api-keys

In [ ]:
from llama_index.readers.web import FireCrawlWebReader

# using firecrawl to crawl a website
firecrawl_reader = FireCrawlWebReader(
    api_key=FIRECRAWL_API_KEY,  # Replace with your actual API key from https://www.firecrawl.dev/
    mode="scrape",
    params={'formats':["markdown", "html"]},
)

# Load documents from a single page URL
documents = firecrawl_reader.load_data(url="https://towardsai.net/")

In [ ]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.node_parser import SentenceSplitter

# To increase chunk size globally
# Settings.chunk_size = 2048  # or even larger like 4096
# Settings.chunk_overlap = 200


# node parser with larger chunk size only for this index
node_parser = SentenceSplitter(
    chunk_size=2048,
    chunk_overlap=200,
)

index = VectorStoreIndex.from_documents(documents, transformations=[node_parser])
query_engine = index.as_query_engine()

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:
res = query_engine.query("What is towards AI aim?")

print(res.response)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['sourceURL'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("-_"*20)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Towards AI aims to make artificial intelligence (AI) accessible to all by offering practical AI Engineering courses, corporate AI bootcamps, and LLM development consultancy for individuals and companies.
-----------------
Node ID	 4dc9135e-c6d5-4150-9689-c2a6c0b3cff5
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8569113365360227
Description	 Master AI with Towards AI. We offer practical AI Engineering courses, corporate AI bootcamps, and LLM development consultancy for individuals and companies.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 afe6e5e9-d5be-4bf3-bbbc-33ee5c9fa2b0
Title	 Towards AI
URL	 https://towardsai.net/
Score	 0.8566070222435507
Description	 Master AI with Towards AI. We offer practical AI Engineering courses, corporate AI bootcamps, and LLM development consultancy for individuals and companies.
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_


# CRAWL A WEBSITE

## Load The CSV

CSV contains the list of tools and url of the page which we use to get information about the tool.

In [ ]:
import requests
import csv

# Google Sheets file URL (CSV export link)
url = 'https://docs.google.com/spreadsheets/d/1gHB-aQJGt9Nl3cyOP2GorAkBI_Us2AqkYnfqrmejStc/export?format=csv'

# Send a GET request to fetch the CSV file
response = requests.get(url)

response_list = []
# Check if the request was successful
if response.status_code == 200:
    # Decode the content to a string
    content = response.content.decode('utf-8')

    # Use the csv.DictReader to read the content as a dictionary
    csv_reader = csv.DictReader(content.splitlines(), delimiter=',')
    response_list = [row for row in csv_reader]
else:
    print(f"Failed to retrieve the file: {response.status_code}")


In [ ]:
import random

start_index = random.randint(0, len(response_list) - 3)
website_list = response_list[start_index:start_index+10] # Crawling 10 websites only.

In [ ]:
import pprint
print("CSV data")
pprint.pprint(website_list)

CSV data
[{'': '',
  'Category': 'Search Library',
  'Company': '',
  'Description': 'Fast, pure Python full text indexing, search, and spell '
                 'checking library',
  'Is a direct URL company /tool website?': 'No',
  'Name': 'Whoosh',
  'Parent': '',
  'Tool Type': 'Library',
  'URL': 'https://pypi.org/project/Whoosh/'},
 {'': '',
  'Category': 'Machine Learning',
  'Company': '',
  'Description': 'An optimized distributed gradient boosting library designed '
                 'to be highly efficient, flexible and portable',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'XGBoost',
  'Parent': '',
  'Tool Type': 'Library',
  'URL': 'https://xgboost.readthedocs.io/en/stable/'},
 {'': '',
  'Category': 'Message Queue',
  'Company': '',
  'Description': 'High-performance asynchronous messaging library',
  'Is a direct URL company /tool website?': 'Yes',
  'Name': 'ZeroMQ',
  'Parent': '',
  'Tool Type': 'Library',
  'URL': 'https://zeromq.org/'},
 {'': '',
  '

## Initialize the Firecrawl

In [ ]:
import os
from firecrawl import FirecrawlApp
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)

In [ ]:
import time

# Crawl websites and handle responses
url_response = {}
crawl_per_min = 1  # Max crawl per minute

# Track crawls
crawled_websites = 0
scraped_pages = 0

for i, website_dict in enumerate(website_list):
    url = website_dict.get('URL')
    print(f"Crawling: {url}")

    try:
        response = app.crawl_url(
            url,
            params={
                'limit': 5,  # Limit pages to scrape per site.
                'scrapeOptions': {'formats': ['markdown', 'html']}
            }
        )
        crawled_websites += 1

    except Exception as exc:
        print(f"Failed to fetch {url} -> {exc}")
        continue

    # Store the scraped data and associated info in the response dict
    url_response[url] = {
        "scraped_data": response.get("data"),
        "csv_data": website_dict
    }

    # Pause to comply with crawl per minute limit for free version its 1 crawl per minute
    if i!=len(website_list) and (i + 1) % crawl_per_min == 0:
        print("Pausing for 1 minute to comply with crawl limit...")
        time.sleep(60)  # Pause for 1 minute after every crawl


Crawling: https://pypi.org/project/Whoosh/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://xgboost.readthedocs.io/en/stable/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://zeromq.org/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://pptr.dev/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://www.nvidia.com/en-us/glossary/bert/#:~:text=BERT%20is%20a%20model%20for,RoBERTa%2C%20ALBERT%2C%20and%20DistilBERT.
Pausing for 1 minute to comply with crawl limit...
Crawling: https://sdv.dev/SDV/user_guides/single_table/ctgan.html
Pausing for 1 minute to comply with crawl limit...
Crawling: https://dalle3.ai/
Pausing for 1 minute to comply with crawl limit...
Crawling: https://wiki.cloudfactory.com/docs/mp-wiki/model-architectures/deeplabv3
Pausing for 1 minute to comply with crawl limit...
Crawling: https://paperswithcode.com/method/efficientdet
Pausing for 1 minute to comply with crawl limit...
Crawling: https://op

## Create  llamaindex documents from the scraped content

In [ ]:
from llama_index.core import Document
documents = []

for _, scraped_content in url_response.items():
    csv_data = scraped_content.get("csv_data")
    scraped_results = scraped_content.get("scraped_data")

    for scraped_site_dict in scraped_results:
        for result in scraped_results:
            markdown_content = result.get("markdown")
            title = result.get("metadata").get("title")
            url = result.get("metadata").get("sourceURL")
            documents.append(
                Document(
                    text=markdown_content,
                    metadata={
                        "title": title,
                        "url": url,
                        "description": csv_data.get("Description"),
                        "category": csv_data.get("Category")
                    }
                )
            )


# Create The RAG Pipeline.

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter

Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=30)

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HT

In [ ]:
from IPython.display import Markdown, display
def display_response(response):
    display(Markdown(f"<b>{response}</b>"))

In [ ]:
query = "Explain GraphRAG" # Enter your query here, it should be relevant to the crawled websites
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>Graph retrieval-augmented generation (GraphRAG) is a framework designed to enhance large language models by organizing fragmented knowledge into structured graphs. This approach addresses the challenges of complex reasoning by improving the way knowledge is constructed and retrieved. The framework integrates various components to optimize performance, particularly in scenarios where domain shifts occur. It utilizes a seed graph schema to guide the extraction of relevant entities and relationships, while also employing community detection techniques to enhance knowledge organization. This results in a more cohesive and efficient reasoning process, allowing for better adaptability and performance across different tasks and domains.</b>

-----------------
Node ID	 1da23efc-071a-43f6-a408-9923e74cdf48
Title	 Trending Papers - Hugging Face
URL	 https://paperswithcode.com/method/efficientdet
Score	 0.5759035989233018
Description	 Scalable and efficient object detection
Category	 Computer Vision
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
Node ID	 3f9313aa-0be7-4953-b725-506e7431c544
Title	 Trending Papers - Hugging Face
URL	 https://paperswithcode.com/method/efficientdet
Score	 0.4738705323473313
Description	 Scalable and efficient object detection
Category	 Computer Vision
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_


In [ ]:
# Querying not relevant to the crawled websites.

query = "What is qdrant?"
res = query_engine.query(query)
display_response(res)

print("-----------------")
# Show the retrieved nodes
for src in res.source_nodes:
  print("Node ID\t", src.node_id)
  print("Title\t", src.metadata['title'])
  print("URL\t", src.metadata['url'])
  print("Score\t", src.score)
  print("Description\t", src.metadata.get("description"))
  print("Category\t", src.metadata.get("category"))
  print("-_"*20)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


<b>The provided information does not include any details about Qdrant. Therefore, I cannot provide an answer regarding what Qdrant is.</b>

-----------------
Node ID	 89cde9fe-5235-4266-bfcb-0ac9203a0a57
Title	 Compare Features | Synthetic Data Vault
URL	 https://docs.sdv.dev/sdv/explore/sdv-enterprise/compare-features
Score	 0.29939159415659344
Description	 Conditional Tabular GAN
Category	 Synthetic Data Generation
-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_
